In [1]:
import os
path_to_frontend = '..'
os.chdir(path_to_frontend)
import q100viz.session as session
import q100viz.devtools as devtools
import q100viz.graphics.graphs as graphs

pygame 2.1.2 (SDL 2.0.16, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html
loading src_points: [[-3.700000000000001, 5.700000000000003], [-2.7000000000000033, 140.60000000000008], [117.69999999999996, 137.8], [117.39999999999998, 1.5000000000000013]]
loading dst_points: [[80, 45], [80, 1035], [1840, 1035], [1840, 45]]


/home/dunland/github/qScope/qScope_frontend/q100viz/gis.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.waermezentrale = read_shapefile(config['WAERMESPEICHER_FILE'], 'Waermespeicher').append(
/home/dunland/github/qScope/qScope_frontend/q100viz/buildings.py:37: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  bestand = bestand.drop('Kataster_S', 1)
/home/dunland/github/qScope/qScope_frontend/q100viz/buildings.py:38: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  bestand = bestand.drop('Kataster_H', 1)


In [ ]:
devtools.select_buildings_for_simulation(session.buildings.df, ['7.31', '7.32', '7.55', '7.56'])

In [ ]:
session.simulation.activate(max_year=2045)

In [ ]:
# import importlib
# importlib.reload(graphs)
graphs.export_combined_emissions(session.buildings.list_from_groups(), "/home/dunland/github/qScope/data/outputs/output_test")

In [ ]:
# importlib.reload(graphs)
graphs.export_combined_energy_prices("/home/dunland/github/qScope/data/outputs/output_test")

In [3]:
df = session.buildings.df[['address', 'spec_heat_consumption', 'avg_spec_heat_consumption', 'type']]
devtools.print_full_df(df)
df.to_csv('buildings_list_export_2022-10_11_16-25.csv')
print(abs(session.buildings.df['spec_heat_consumption'] - session.buildings.df['avg_spec_heat_consumption']).describe())

                     address  spec_heat_consumption  \
id                                                    
6.07  Hans-Böckler-Straße 26                170.000   
3.05      Stettiner Straße 7                364.956   
4.08          Neue Heimat 15                166.800   
3.12     Stettiner Straße 18                136.239   
6.06  Hans-Böckler-Straße 24                533.038   
6.01  Hans-Böckler-Straße 14                170.000   
3.10     Stettiner Straße 14                 85.183   
4.31          Neue Heimat 38                113.245   
4.19          Neue Heimat 14                221.357   
2.16            Im Redder 12                118.796   
2.06            Im Redder 11                130.869   
4.01           Neue Heimat 1                281.196   
2.08            Im Redder 15                241.703   
4.12          Neue Heimat 23                198.022   
6.08  Hans-Böckler-Straße 28                 88.089   
2.04             Im Redder 7                134.673   
4.24      